In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("train.csv")

###EDA (제거여부)

  - id: 칼럼id -제거-
  - base_date: 날짜     -제거-
  - day_of_week: 요일         라벨로 변경
  - base_hour: 시간대
  - lane_count: 차로수
  - road_rating: 도로 등급       (106:지방도,103:일반국도,107:시도,군도)로 구성
  - road_name: 도로 이름
  - multi_linked: 중용구간 여부(0,1)로 구성  ->0개수:4698978 1개수:2239 -제거-
  - connect_code: 연결로 코드         (0,103)로 구성
  - maximum_speed_limit: 최고속도 제한       타겟과의 상관 관계가 제일 높음
  - vehicle_restricted: 통과제한 차량   (값이 한개임) -제거-
  - weight_restricted: 통과제한 하중         타겟과의 상관 관계가 두번째로 높음
  - height_restricted: 통과 제한 높이    (값이 한개임) -제거-
  - road_type: 도로 유형                     (0,3) 으로 구성 타겟과의 상관 관계가 세번재로 높음
  - start_node_name: 시작지점명            온핫 인코딩 사용?
  - start_latitude: 시작지점 위도
  - start_longitude: 시작지점 경도
  - start_turn_restricted: 시작 지점의 회전제한 유무      (있음,없음)으로 구성
  - end_node_name: 도착지점명             온핫 인코딩 사용?
  - end_latitude: 도착지점 위도
  - end_longitude: 도착지점 경도
  - end_turn_restricted: 도착 지점의 회전제한 유무    (있음,없음)으로 구성
  - target: 평균속도(km)


  1. 참조 https://www.law.go.kr/LSW/admRulInfoP.do?admRulSeq=2100000157569  (도로 규정)
  2. 참조 https://tttsss77.tistory.com/147 (위도, 경도로 지도 찾기)
  3. 참조 https://velog.io/@eodud0582/Folium 지도위에 표시(위도,경도 이용)
  4. 참조 https://bskyvision.com/entry/python-%EC%A3%BC%EC%86%8C%EB%A5%BC-%EC%9C%84%EB%8F%84-%EA%B2%BD%EB%8F%84-%EC%A2%8C%ED%91%9C%EB%A1%9C-%EB%B0%94%EA%BF%94%EC%A3%BC%EB%8A%94-Geocoder-API-%EC%82%AC%EC%9A%A9%EB%B2%95 도로명 주소로 (위도,경도 찾기)

In [ ]:
df.isnull().sum().plot(kind="bar")   ##null값 없음

In [ ]:
#상관관계도 확인
plt.figure(figsize = (15,8))
sns.heatmap(df.corr(), annot=True)
###최고 속도 제한이 제일 높음
###통과제한 하중 높이가 두번재
###도로 유형이 3번째
### 3개의 칼럼 확인

In [ ]:
##너무 높은값 제거
sns.boxplot(x = "maximum_speed_limit", y = "target", data = df)


###전처리

In [ ]:
##필요없는 칼럼 삭제
df.drop(["id", "base_date","vehicle_restricted", "height_restricted"], axis=1, inplace=True)

In [ ]:
# 1차 자료형 (변형  혹시 데이터들이 너무 클경우: 64->32로 변형)
# to_int32 = ["base_hour", "lane_count", "road_rating", "multi_linked", "connect_code", "road_type"]
# to_float32 = ["maximum_speed_limit", "weight_restricted", "target"]

# for i in to_int32:
#     df[i] = df[i].astype("int32")
# for j in to_float32:
#     df[j] = df[j].astype("float32")

In [ ]:
df2 = df[df["target"] <100].copy() ##속도가 100 넘는값들 제외하고 df2에 데이터 프레임 저장

In [ ]:
from sklearn.preprocessing import LabelEncoder ##라벨 인코딩위해 불러오기

In [ ]:
chage_c = ["day_of_week","road_name", "start_node_name", "end_node_name",
           "start_turn_restricted", "end_turn_restricted", "weight_restricted", "road_rating",
           "road_type",]
for i in chage_c:
    le = LabelEncoder()
    le=le.fit(df2[i])
    df2[i]=le.transform(df2[i])

In [ ]:
# from sklearn.preprocessing import OneHotEncoder  (원-핫 인코딩이필요한 경우)

# ohe = OneHotEncoder (sparse = False)

###훈련셋&시험셋 나누기


In [ ]:
from sklearn.model_selection import train_test_split ###불러오기


In [ ]:
X = df.drop('target', axis=1)           ##x축 
y = df.target                          ##y축

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=100
)

##랜덤포레스트 모델: 김영광

In [ ]:
from sklearn.ensemble import RandomForestRegressor  ##랜덤포레스트 불러오기

In [ ]:
model = RandomForestRegressor(random_state=100)
model.fit(X_train, y_train)
train_pred = model.predict(X_train)
test_pred = model.predict(X_test) 

###라이트GBM : 이준희


In [ ]:
import lightgbm as lgb

In [ ]:
model = lgb.LGBMRegressor(random_state=100)
model.fit(X_train, y_train)
pred = model.predict(X_test)

###*CatBoost :배진혁

In [ ]:
pip install catboost ###캣부스트 설치
import catboost as cat
from catboost import CatBoostRegressor            ###모델링 구현
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
cat = CatBoostRegressor(iterations=1000,
                              learning_rate=0.05,
                              depth=10,
                              eval_metric='RMSE',
                              random_seed = 42,
                              bagging_temperature = 0.2,
                              od_type='Iter',
                              metric_period = 50,
                              od_wait=20)
cat.fit(X_train,y_train)
cat_predict = cat.predict(X_test)

###XGBoost

In [ ]:
import xgboost as xgb   ##XGBoost 불러오기
xgb = xgb.XGBRegressor(n_estimators=100, max_depth = 100)  ##모델링
xgb.fit(X_train, y_train)
xgb_predict = xgb.predict(X_test)

###결정트리

In [ ]:
from sklearn.tree import DecisionTreeRegressor        ##DecisionTreeRegressor불러오기
dtree = DecisionTreeRegressor (max_depth = 15,
                               min_samples_split=2,
                               min_samples_leaf=1,
                               min_weight_fraction_leaf=0.0,
                               max_features=None,
                               random_state=1,
                               max_leaf_nodes=None,
                               min_impurity_decrease=0.0,
                               ccp_alpha=0)
dtree.fit(X_train,y_train)
dtree_predict = dtree.predict(X_test)

In [ ]:
##최적화 트리 깊이 확인
def test_depth(depth):
    # max_depth : 트리의 깊이 제한
    model = DecisionTreeRegressor(random_state=100, max_depth=depth)
    model.fit(X_train, y_train) # 훈련 데이터 -> 모델 학습
    # 훈련 데이터로 모델을 통해서 예측
    train_pred = model.predict(X_train) # 학습을 시킨 값으로 다시 예측
    # 시험 데이터로 모델을 통해서 예측
    test_pred = model.predict(X_test) 
    print(f"depth : {depth}")
    print(f"Train Score : {model.score(X_train,y_train)}")
    print(f"Test Score : {model.score(X_test,y_test)}") 

###하이퍼 파라미터 && Kfold

In [ ]:
train_rmse_total = []
test_rmse_total = []

for train_index, test_index in kf.split(X): # 반복 <- 5기준으로 나뉜 K-Fold
    X_train, X_test = X.loc[train_index], X.loc[test_index] # 독립변수들의 훈련셋/시험셋
    y_train, y_test = y[train_index], y[test_index] # 종속변수의 훈련셋/시험셋

    model = RandomForestRegressor(random_state=100) ##특정 알고리즘##
    (         
        n_estimators = 500,
        max_depth = 60,
        min_samples_split = 5,
        min_samples_leaf = 1,
        n_jobs= -1,
        random_state = 100
    )

    model.fit(X_train, y_train) # 학습
    train_pred = model.predict(X_train) # 훈련셋 예측 (잘 학습되었나? 과최적화, 오버피팅?)
    test_pred = model.predict(X_test) # 시험셋 예측 (잘 예측하나?, 언더피팅)

###에러(RMSE) 스코어 확인

In [ ]:
print(
    # 실제값, 예측값 -> 에러를 비교
    "train_rmse : ", sum(train_rmse_total) / len(train_rmse_total),
    "test_rmse : ", sum(test_rmse_total) / len(test_rmse_total)
)

In [ ]:
print(f"Train Score : {model.score(X_train, y_train)}") # 스코어 확인
print(f"Test Score : {model.score(X_test, y_test)}") 